In [1]:
from numpy.linalg import eig
import numpy as np
import cv2
import os
import sys
import pickle

In [2]:
configuration_file = 'config/part_1.cfg'

In [ ]:
**Main: process_video.py**

In [ ]:
from process_video import*
from src.extract_features import *
from src.matching_features import *
from src.homography import *
from src.ransac import *
from src.parsing import *
from src.display_video import *
from src.view_homography import *
from src.outputs import *
from src.optimized_corners import *

config_data = parse_configuration_file(configuration_file) #Parse the configuration file
match_img1 , match_map = parse_points(config_data) #Parse the points from the configuration file
video_path, type_homography, file_name_keypoints, file_name_tranformations = final_parsing(config_data)

sift_points, nr_points = extract_features(video_path)

create_output_keypoints(sift_points, file_name_keypoints, nr_points)
    

**Main: compute_transforms.py**

In [3]:
from src.extract_features import *
from src.parsing import *
from src.display_video import *
from process_video import *
from compute_transform import*

config_data = parse_configuration_file(configuration_file) #Parse the configuration file
match_img1 , match_map = parse_points(config_data) #Parse the points from the configuration file
video_path, type_homography, file_name_keypoints, file_name_tranformations, \
    width, height = final_parsing_compute_transform(config_data)

H_frame1_to_map =compute_homography(match_img1, match_map)    
sift_points, nr_points = extract_keypoints(file_name_keypoints)
print("width, height", width, height)
match = matching_features_SCIKITLEARN(sift_points)
H_sequential = create_sequential_homographies(match, sift_points)



image matches:  [('225', '131'), ('580', '120'), ('626', '305'), ('133', '303')]
map matches:  [('225', '131'), ('580', '120'), ('626', '305'), ('133', '303')] 

width, height 804 454


In [8]:
H_all = all_homographies(H_sequential,height, width, sift_points)
H_tomap,Htoframe1 = homography_to_map(H_sequential, H_frame1_to_map,height, width, sift_points)


Entered cycle, indexes: 2 0
Entered cycle, indexes: 3 0
Entered cycle, indexes: 4 0
Entered cycle, indexes: 5 0
Entered cycle, indexes: 6 0
Entered cycle, indexes: 7 0
Entered cycle, indexes: 8 0
Entered cycle, indexes: 9 0
Entered cycle, indexes: 10 0
Entered cycle, indexes: 11 0
Entered cycle, indexes: 12 0
Entered cycle, indexes: 13 0
Entered cycle, indexes: 14 0
Entered cycle, indexes: 15 0
Entered cycle, indexes: 16 0
Entered cycle, indexes: 17 0
Entered cycle, indexes: 18 0
Entered cycle, indexes: 19 0


Output of process_video.py:

In [6]:
from scipy.io import loadmat

def extract_keypoints(file_path):
    data = loadmat(file_path)
    keypoint_data = data['Keypoints']
    kp_list = []
    nr_points = None

    for item in keypoint_data:
        points = item.reshape((item.shape[1],item.shape[0])) # Transpose to revert the reshape operation
        kp_list.append(points)
        # Store the number of points (assuming all arrays have the same size)
        nr_points = points.shape[1] if nr_points is None else nr_points
    return kp_list, nr_points

sift_points_extracted, nr_points_extracted = extract_keypoints(file_name_keypoints)


Output of compute_transform.py:

In [8]:
from src.outputs import create_output
i = create_output(H_output, file_name_tranformations)        



**Display results:**

In [5]:
def show_pixel_coordinates(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Clicked at (x={x}, y={y})")

def display_frame(name,frame):
    cv2.namedWindow(name)
    cv2.setMouseCallback(name, show_pixel_coordinates)
    cv2.imshow(name, frame)

def display(frame1, frame2,homography_de2_para1 , idx1, idx2 ):
    H= homography_de2_para1.reshape((3,3))
    
    height, width = frame2.shape[:2]
    # Apply homography to frame 2
    warped_frame2 = cv2.warpPerspective(frame2, H, (width, height))

    while True:
        display_frame(f'Frame {idx1}', frame1)

        display_frame(f'Frame{ idx2} transformed with homography to coordinates of frame {idx1}', warped_frame2)

        display_frame(f"Frame {idx2}", frame2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        elif cv2.waitKey(1) & 0xFF == ord('e'):
            return True
    cv2.destroyAllWindows()
    return False

def extract__frames(video_path):
    """Extracts the features from the video and stores them in a list"""
    print(video_path)
    capture = cv2.VideoCapture(os.path.abspath(video_path))
    k = 0
    frames=[]
    count_frames(video_path)
    while k <= 1900:
        capture.set(cv2.CAP_PROP_POS_FRAMES, k)
        success, frame = capture.read() #read the video
        if success:
            frames.append(frame)
        k += 100
    return frames

In [6]:

frames= extract__frames('video/trymefirst_lisbon.mp4')

video/trymefirst_lisbon.mp4
Total frames of the video:  1901


In [9]:
#to test homography to map function (to frame 0 and not to map)
img1=frames[0]
idx1 = 0
print('Press q to change image\nPress e to exit')

for i in range(1,len(frames)-1):
    idx2 = i 
    img2= frames[i]
    exit_bool = display(img1, img2, Htoframe1[2:,i], idx1, idx2 )
    if exit_bool:
        cv2.destroyAllWindows()
        break

Press q to change image
Press e to exit
Clicked at (x=383, y=210)
Clicked at (x=394, y=222)
Clicked at (x=467, y=181)
Clicked at (x=463, y=129)
Clicked at (x=474, y=125)
Clicked at (x=634, y=145)
Clicked at (x=146, y=223)
Clicked at (x=263, y=200)
Clicked at (x=223, y=143)
Clicked at (x=196, y=174)
Clicked at (x=305, y=152)
Clicked at (x=225, y=191)
Clicked at (x=240, y=156)
Clicked at (x=254, y=160)
Clicked at (x=283, y=178)
Clicked at (x=552, y=90)
Clicked at (x=273, y=113)
Clicked at (x=331, y=138)
Clicked at (x=190, y=314)
Clicked at (x=542, y=174)
Clicked at (x=172, y=173)
Clicked at (x=521, y=98)
Clicked at (x=275, y=215)
Clicked at (x=467, y=110)
Clicked at (x=190, y=152)
Clicked at (x=372, y=301)
Clicked at (x=345, y=126)
Clicked at (x=473, y=148)


In [7]:
#to test homography to each other
print('Press q to change image\nPress e to exit')
#H_output = all_homographies(H_sequential)

for i in range(H_all.shape[1]):
    idx1= int(H_all[0,i])
    idx2= int(H_all[1,i])

    img1 = frames[ idx1-1 ] 
    img2= frames[  idx2-1]
    exit_bool = display(img1, img2, H_all[2:,i], idx1, idx2 )
    if exit_bool:
        cv2.destroyAllWindows()
        break

Press q to change image
Press e to exit
Clicked at (x=520, y=270)
Clicked at (x=655, y=224)
Clicked at (x=609, y=222)
Clicked at (x=376, y=162)
Clicked at (x=162, y=146)
Clicked at (x=373, y=111)
Clicked at (x=349, y=128)
Clicked at (x=377, y=116)
Clicked at (x=392, y=142)
Clicked at (x=397, y=129)
Clicked at (x=368, y=222)
Clicked at (x=513, y=320)
Clicked at (x=409, y=216)
Clicked at (x=421, y=103)
Clicked at (x=371, y=129)
Clicked at (x=535, y=236)
Clicked at (x=436, y=123)
Clicked at (x=386, y=175)
Clicked at (x=362, y=74)
Clicked at (x=443, y=83)
Clicked at (x=403, y=206)
Clicked at (x=362, y=160)
Clicked at (x=436, y=128)
Clicked at (x=377, y=146)
Clicked at (x=554, y=102)
Clicked at (x=638, y=161)
Clicked at (x=447, y=105)
Clicked at (x=403, y=82)
Clicked at (x=184, y=96)
Clicked at (x=180, y=166)
Clicked at (x=212, y=107)
Clicked at (x=401, y=166)
Clicked at (x=310, y=149)
Clicked at (x=261, y=132)
Clicked at (x=617, y=190)
Clicked at (x=366, y=329)
Clicked at (x=340, y=303)
Cl